In [1]:
import numpy as np
import pandas as pd
import os
import gcsfs
import xarray as xr
from functools import partial
from IPython.display import display
from glob import glob
import warnings
import datetime
import pprint

import configparser
config = configparser.ConfigParser()

In [2]:
from esgf import esgf_search_sites, search, esgf_search

### Load specified `zcs_id` from config file as well as I/O and ESGF specs

In [3]:
config.read('config.cfg')
zcs_dict = {}
zcs_dict['activity_id'] = (config.get('zcs_id', 'activity_id'))
zcs_dict['institution_id'] = config.get("zcs_id", "institution_id")
zcs_dict['source_id'] = config.get("zcs_id", "source_id")
zcs_dict['experiment_id'] = config.get("zcs_id", "experiment_id")
zcs_dict['member_id'] = config.get("zcs_id", "member_id")
zcs_dict['table_id'] = config.get("zcs_id", "table_id")
zcs_dict['variable_id'] = config.get("zcs_id", "variable_id")
zcs_dict['grid_label'] = config.get("zcs_id", "grid_label")
# zcs_id = (activity_id, institution_id, source_id, experiment_id, member_id, table_id, variable_id, grid_label)

zarr_local = config.get("FILEPATHS", "zarr_local")
update_ESGF = config.getboolean("ESGF", "update_ESGF")
local_node = config.getboolean("ESGF", "local_node")

In [4]:
fs = gcsfs.GCSFileSystem(token='anon', access='read_only')
dtype = esgf_search_sites()
print('possible ESGF API search nodes: ',list(dtype.keys()))
node = 'llnl'
print("setting current ESGF node to %s" %node)
ESGF_site = dtype[node]

possible ESGF API search nodes:  ['llnl', 'ipsl', 'nci', 'ceda', 'gfdl', 'dkrz']
setting current ESGF node to llnl


### get what exists in GCS as zarr stores already 

In [ ]:
dzLocal = pd.read_csv('https://cmip6.storage.googleapis.com/cmip6-zarr-consolidated-stores-noQC.csv', dtype={'version': 'unicode'})
# len(dzLocal),len(dESGF)
len(dzLocal)

In [ ]:
for key in zcs_dict:
    if zcs_dict[key] != 'All':
        dzLocal = dzLocal.loc[(dzLocal[key] == zcs_dict[key])]
len(dzLocal)
pprint.pprint(zcs_dict)
print("there are currently %.0f models in GCS for the above zarr catalog store unique id" %len(dzLocal))

### get what exists in ESGF that is not in GCS 

In [8]:
if update_ESGF:
    # get for all sources and members, filter request later
    df_list = []
    experiment_id = zcs_dict['experiment_id']
    variable_id = zcs_dict['variable_id']
    table_id = zcs_dict['table_id']
    print(experiment_id,variable_id)
    
    files = esgf_search(server=ESGF_site, mip_era='CMIP6', variable_id=variable_id, 
                           table_id=table_id, experiment_id=experiment_id, 
                           page_size=500, verbose=False, local_node=False)
    print('got-em')

    files.loc[:,'version'] = [str.split('/')[-2] for str in files['HTTPServer_url']]
    files.loc[:,'file_name'] = [str.split('/')[-1] for str in files['HTTPServer_url']]
    # might need to set activity_id to activity_drs for some files (see old versions)
    files.loc[:,'activity_id'] = files.activity_drs
    df_list += [files.drop_duplicates(subset =["file_name","version","checksum"]) ]

    dESGF = pd.concat(df_list,sort=False)
    dESGF = dESGF.drop_duplicates(subset =["file_name","version","checksum"])
    dESGF.to_csv(os.path.join(zarr_local, 'ESGF_specific.csv'),index=False)
else:
    dESGF = pd.read_csv('csv/ESGF_specific.csv')
    dESGF = dESGF.drop_duplicates(subset =["file_name","version","checksum"])

ssp370 tasmax
got-em


In [ ]:
'''if update_ESGF:
    # get for all sources and members, filter request later
    df_list = []
    for experiment_id in zcs_dict['experiment_id']:
        for variable_id in zcs_dict['variable_id']:
            print(experiment_id,variable_id)
            try:
                files= esgf_search(server=ESGF_site, mip_era='CMIP6', variable_id=variable_id, 
        table_id=table_id, experiment_id=experiment_id, page_size=500, verbose=False, local_node=False)
                #print('got-em')
            except:
                #print(experiment_id, table_id,variable_id,'nothing doing')
                continue

            files.loc[:,'version'] = [str.split('/')[-2] for str in files['HTTPServer_url']]
            files.loc[:,'file_name'] = [str.split('/')[-1] for str in files['HTTPServer_url']]
            # might need to set activity_id to activity_drs for some files (see old versions)
            files.loc[:,'activity_id'] = files.activity_drs
            df_list += [files.drop_duplicates(subset =["file_name","version","checksum"]) ]

    dESGF = pd.concat(df_list,sort=False)
    dESGF = dESGF.drop_duplicates(subset =["file_name","version","checksum"])
    dESGF.to_csv('csv/ESGF_specific.csv',index=False)
else:
    dESGF = pd.read_csv('csv/ESGF_specific.csv')
    dESGF = dESGF.drop_duplicates(subset =["file_name","version","checksum"])'''

In [ ]:
urls = get_dataset_urls(zcs_id)